# Analisis de Datos Exploratorio
#### para el Club de Datos 


La Geografía del Crimen  
  
    - Buscar información sobre los distritos, sus características, qué lugares hay (casas, oficinas, fábricas, etc.), historia.  
    - ¿Cuáles son los  más comunes por distrito?  
    - ¿Por qué hay distritos con mucho más crimen que otros? ¿Con qué información demográfica se podría cruzar? Buscar otros datasets.  
    - Analizar las calles más peligrosas. ¿Son avenidas, calles internas? ¿Qué distritos tocan?  
    - ¿En qué parte de la calle ocurren ciertos crimenes? ¿A mitad de cuadra, en la esquina?  
    - Hay algunas calles más largas que otras. ¿Se podrán "normalizar" los datos para descubrir la calle con mayor "densidad" de crimen?  
    - En el mismo sentido, cuando calculamos la densidad en los distritos, dividimos por el área. ¿Capaz sería más razonable/interesante dividir por la población de ese distrito?  
    - Buscar otros sitios geográficos de interés. Lineas/estaciones de subte, el puente, comisarías, otros edificios importantes.  
  
La  Resolución de Crimenes
  
    - ¿Cómo es la resolución de casos? Analizar por distrito, categoría de crímen, día de la semana.  
    - Hacer algún modelo que dado ciertos datos sobre un crimen, prediga la probabilidad de que se resuelva.  

In [2]:
# Objetivo : Importar las librerias necesarias para realizar el analisis

import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import requests
from bs4 import BeautifulSoup

In [5]:
# Objetivo : Cargar el set de datos

# Leo los datos con Geopandas
datoSanMartin =  geopandas.read_file("san_martin_crime.geojson")
# Muestreo el dataset obtenido
datoSanMartin.head()

,id,dependency_id,dependency_name,creation_date,felony_id,felony_name,city_name,municipality_name,geometry
0,304310,1746,CRIA.SAN MARTIN 4TA,2019-07-01T00:33:06.390000-03:00,20372,Robo Agravado (vehículo dejado en la vía públi...,Jose Leon Suarez,San Martín,POINT (-58.57750 -34.53950)
1,304344,1725,CRIA.SAN MARTIN 2DA,2019-07-01T02:17:53.705002-03:00,75,Robo - Art.164,Villa Ballester,San Martín,POINT (-58.55320 -34.54890)
2,304385,1742,CRIA.SAN MARTIN 3RA,2019-07-01T07:21:15.085000-03:00,20122,Robo agravado (uso de armas de fuego) - Art.16...,Villa Lynch,San Martín,POINT (-58.52480 -34.59040)
3,304626,2085,EST.POL.COM.CASTELLI,2019-07-01T11:37:22.268999-03:00,17,Lesiones leves - Art.89,Castelli,Castelli,POINT (-58.52330 -34.56510)
4,304633,1725,CRIA.SAN MARTIN 2DA,2019-07-01T11:40:24.999001-03:00,20128,Robo agravado (comisión con efracción) - Art.1...,Villa Ballester,San Martín,POINT (-58.55790 -34.53060)


### Caracteristicas Principales del dataset

In [6]:
# Objetivo : Mostrar las dimensiones del dataset
datoSanMartin.shape

(9911, 9)

In [7]:
# Objetivo : Mostrar los tipos de datos de cada columna
datoSanMartin.dtypes

id                      int64
dependency_id           int64
dependency_name        object
creation_date          object
felony_id               int64
felony_name            object
city_name              object
municipality_name      object
geometry             geometry
dtype: object

In [8]:
# Objetivo : Mostrar la cantidad de nulos en cada columna
datoSanMartin.isnull().sum()

id                   0
dependency_id        0
dependency_name      0
creation_date        0
felony_id            0
felony_name          0
city_name            0
municipality_name    0
geometry             0
dtype: int64

In [9]:
# Objetivo: Ver que municipios hay y cuales son los que mas delitos tienen

datoSanMartin['municipality_name'].value_counts(dropna=False)

San Martín         9290
Tres De Febrero     145
Vicente López        89
La Matanza           34
Merlo                33
                   ... 
Lobos                 1
Rojas                 1
Carmen De Areco       1
Daireaux              1
Tandil                1
Name: municipality_name, Length: 83, dtype: int64

### Enriquecimiento de los datos

In [10]:
# Objetivo: Buscar información sobre los distritos, sus características, qué lugares hay (casas, oficinas, fábricas, etc.), historia.

# Defino el url al que voy a hacer el request
url = 'https://es.wikipedia.org/wiki/Anexo:Partidos_de_la_provincia_de_Buenos_Aires'
# Hago el request para obtener el HTML de la pagina donde se encuentran los datos que queremos sumar
html = requests.get(url)
# Utilizo Beautiful para poder parsear el HTML y acceder a la informacion mas facil
soup = BeautifulSoup(html.text, 'html.parser')

In [11]:
# Objetivo: Construir el set de datos para mergear con el dataframe original.

# Extraigo la tabla que tiene los datos
table = soup.find_all('table')[1]

# Extraigo los datos en las filas de la tabla y las guardo en la lista
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)

# Creo el dataframe para luego mergearlo con el original
df = pd.DataFrame(output_rows)

# Extraigo los titulos y los guardo en la lista
titles = []
for title in table.find_all('th'):
    titles.append(title.text)

# Le asigno los titlos al dataframe
df.columns = titles

df.head()

,Escudo\n,Partido,Sección electoral,Códigos INDEC:divisiones político-administrativas/ gobiernos locales[2]​[3]​,Cabecera,Año de creación[4]​,Origen,Intendente/a (2019-2023),Población (2010)[5]​,Superficie (km²)[6]​,Densidad (hab./km²),Mapa,Número de concejales[7]​[8]​,Consejeros escolares[9]​\n
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,\n,Adolfo Alsina,6.ª,06007/ 060007,Carhué,1886,Por ley provincial 1827,Javier Andrés (Juntos por el Cambio),17 072,5878,"2,90",,12,6\n
2,\n,Adolfo Gonzales Chaves,6.ª,06014/ 060014,Adolfo Gonzales Chaves,1916,Por Ley provincial 3632 – Con tierras de los ...,Eduardo Marcelo Santillán (Frente de Todos),12 047,3859,"3,12",,12,4\n
3,\n,Alberti,4.ª,06021/ 060021,Alberti,1910,Por ley provincial 3237 - Con tierras de los p...,Germán Lago (Frente de Todos),10 654,1130,"9,43",,12,4\n
4,\n,Almirante Brown,3.ª,06028/ 060028,Adrogué,1873,Por ley provincial 856 - Con tierras de los pa...,Mariano Cascallares (Frente de Todos),552 902,129,"4286,06",,24,10\n


In [12]:
# Objetivo: Preparar el dataset para ser mergeado

# Proceso y cambio el tipo de dato de las columnas numericas

# Año de creacion
df.loc[df['Año de creación[4]​'].isnull(), :] = 0
df['Año de creación[4]​'].replace('', '0', regex=True, inplace=True)
df['Año de creación[4]​'] = df['Año de creación[4]​'].astype(int)

# Poblacion
df.loc[df['Población (2010)[5]​'].isnull(), :] = 0
df['Población (2010)[5]​'].replace('\xa0', '', regex=True, inplace=True)
df['Población (2010)[5]​'].replace(' ', '', regex=True, inplace=True)
df['Población (2010)[5]​'].replace('IncluidoenelpartidodeChascomús', '0', regex=True, inplace=True)
df['Población (2010)[5]​'].replace('.IncluyealpartidodeLezama', '', regex=True, inplace=True)
df['Población (2010)[5]​'].replace('', '0', regex=True, inplace=True)
df['Población (2010)[5]​'] = df['Población (2010)[5]​'].astype(int)

# Superficie
df.loc[df['Superficie (km²)[6]​'].isnull(), :] = 0
df['Superficie (km²)[6]​'].replace(',', '', regex=True, inplace=True)
df['Superficie (km²)[6]​'].replace('\u200b', '', regex=True, inplace=True)
df['Superficie (km²)[6]​'].replace('[10]', '', regex=True, inplace=True)
df['Superficie (km²)[6]​'].replace('', '0', regex=True, inplace=True)
df['Superficie (km²)[6]​'].replace('2[]', '2', inplace=True)
df['Superficie (km²)[6]​'].replace('\xa0625\xa084', '', regex=True, inplace=True)
df['Superficie (km²)[6]​'] = df['Superficie (km²)[6]​'].astype(int)

# Densidad Poblacional
df.loc[df['Densidad (hab./km²)'].isnull(), :] = 0
df['Densidad (hab./km²)'].replace('\u200b', '', regex=True, inplace=True)
df['Densidad (hab./km²)'].replace('\xa0637', '', regex=True, inplace=True)
df['Densidad (hab./km²)'].replace(',', '', regex=True, inplace=True)
df['Densidad (hab./km²)'] = df['Densidad (hab./km²)'].astype(float)

# Numero de consejales
df.loc[df['Número de concejales[7]​[8]​'].isnull(), :] = 0
df['Número de concejales[7]​[8]​'].replace('', '0', regex=True, inplace=True)
df['Número de concejales[7]​[8]​'] = df['Número de concejales[7]​[8]​'].astype(int)


# Nombres de las columnas
df.rename(
    columns = {
        "Año de creación[4]​": "anoCreacion", 
        "Población (2010)[5]​": "poblacion", 
        "Superficie (km²)[6]​": "superficieKM2",
        "Densidad (hab./km²)": "densidadPoblacionalHabKM2",
        "Número de concejales[7]​[8]​": "cantConsejales",
        "Partido": "nombrePartido"
        }, 
    inplace=True
    )

In [13]:
partidos = df[["nombrePartido","anoCreacion", "poblacion", "superficieKM2", "densidadPoblacionalHabKM2", "cantConsejales"]]

In [14]:
# Objetivo: mergear los sets de datos

#Validacion inicial de las dimensiones
print('Antes de procesar: El set de datos tiene {} registros'.format(datoSanMartin.shape[0]))

datoSanMartinMerged = datoSanMartin.merge(partidos, how='left', left_on='municipality_name', right_on='nombrePartido')

#Validacion final de las dimensiones
print('Despues de procesar: El set de datos tiene {} registros'.format(datoSanMartinMerged.shape[0]))

Antes de procesar: El set de datos tiene 9911 registros
Despues de procesar: El set de datos tiene 9911 registros


## Respondemos Preguntas

- ¿Cuáles son los  más comunes por distrito? 

In [15]:
# Objetivo: Responder la pregunta de arriba, y dejar un dataset donde preguntemos por el barrio
agrupedCrimeData = datoSanMartin.groupby(by=['municipality_name', 'felony_name']).count().sort_values(by=['municipality_name', 'felony_name'])['id']

agrupedCrimeData['Adolfo Alsina']

felony_name
Averiguación de Ilícito    1
Daño - Art.183             1
Name: id, dtype: int64

- ¿Por qué hay distritos con mucho más crimen que otros? ¿Con qué información demográfica se podría cruzar? Buscar otros datasets.  
    - Analizar las calles más peligrosas. ¿Son avenidas, calles internas? ¿Qué distritos tocan?  
    - ¿En qué parte de la calle ocurren ciertos crimenes? ¿A mitad de cuadra, en la esquina?  
    - Hay algunas calles más largas que otras. ¿Se podrán "normalizar" los datos para descubrir la calle con mayor "densidad" de crimen?  
    - En el mismo sentido, cuando calculamos la densidad en los distritos, dividimos por el área. ¿Capaz sería más razonable/interesante dividir por la población de ese distrito?  
    - Buscar otros sitios geográficos de interés. Lineas/estaciones de subte, el puente, comisarías, otros edificios importantes.

In [16]:
# Objetivo: Responder la pregunta de arriba, y 
agrupedCrimeAnalisisData = datoSanMartin.groupby(by=['municipality_name']).count()

agrupedCrimeAnalisisData

,id,dependency_id,dependency_name,creation_date,felony_id,felony_name,city_name,geometry
municipality_name,,,,,,,,
Adolfo Alsina,2,2,2,2,2,2,2,2
Alberti,1,1,1,1,1,1,1,1
Almirante Brown,9,9,9,9,9,9,9,9
Arrecifes,2,2,2,2,2,2,2,2
Avellaneda,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...
Tres De Febrero,145,145,145,145,145,145,145,145
Tres Lomas,1,1,1,1,1,1,1,1
Vicente López,89,89,89,89,89,89,89,89


In [17]:
# Objetivo: mergear los sets de datos

#Validacion inicial de las dimensiones
print('Antes de procesar: El set de datos tiene {} registros'.format(datoSanMartin.shape[0]))

datoSanMartinMerged = datoSanMartin.merge(partidos, how='left', left_on='municipality_name', right_on='nombrePartido')

#Validacion final de las dimensiones
print('Despues de procesar: El set de datos tiene {} registros'.format(datoSanMartinMerged.shape[0]))

Antes de procesar: El set de datos tiene 9911 registros
Despues de procesar: El set de datos tiene 9911 registros
